In [727]:
import sys
import os

code_dir = os.path.abspath(os.path.join(os.getcwd(), "..", "code"))
sys.path.append(code_dir)

import json
import numpy as np
from evaluation.metrics_real_data import compute_all_utilities
from anonymization.utils import compute_block_probability_matrix, generate_anonymized_graph

In [728]:
n = 100  # Change before run
K = 15   # Change before run
seed = 2025 # Change before run
dataset = "ca-hepth"  # "enron", "ca-hepth", "facebook"
algorithm = "swap"   # "swap", "merge", "hybrid"
filename_suffix = f"n{n}_K{K}_seed{seed}"

A = np.load(f"../data/real/{dataset}/adjacency_A_{dataset}_n{n}_seed{seed}.npy")
z_anonymized = np.load(f"../data/results/real/{dataset}/{algorithm}/z_anonymized_{algorithm}_{filename_suffix}.npy").tolist()
metrics_dir = f"../data/results/real/{dataset}/metrics"
os.makedirs(metrics_dir, exist_ok=True)

In [729]:
#Loading and reconstructing
B_anonymized, _ = compute_block_probability_matrix(A, z_anonymized)
A_hat = generate_anonymized_graph(z_anonymized, B_anonymized, seed=seed)
np.save(f"../data/results/real/{dataset}/{algorithm}/adj_matrix_{filename_suffix}.npy", A_hat)

In [730]:
# Compute metrics (A_hat is the anonymized graph)
results = compute_all_utilities(A_hat, z_anonymized=z_anonymized, A_original=A)

In [731]:
# Print results
for k, v in results.items():
    print(f"{k}: {v}")

degree_distribution_difference: 0.12884098726725127
clustering_coefficient: 0.18330014430014432
shortest_path_mean: None
shortest_path_std: None
assortativity: 0.4285124321541198
modularity_anonymized: 0.6098807304418139
spectral_eigenvalues: [7.69540165 5.9750315  3.77453538 3.68683603 3.48861471 3.16597622
 3.03131542 2.90230909 2.65800489 2.47040405]


In [732]:
# Before saving converting arrays to lists so JSON can handle it
for key, value in results.items():
    if isinstance(value, np.ndarray):
        results[key] = value.tolist()

output_path = f"{metrics_dir}/utilities_{algorithm}_{filename_suffix}.json"
with open(output_path, "w") as f:
    json.dump(results, f, indent=4)

print(f"\n Log: Saved utility results to {output_path}")


 Log: Saved utility results to ../data/results/real/ca-hepth/metrics/utilities_swap_n100_K15_seed2025.json
